<a href="https://colab.research.google.com/github/Qays-H/SSL-CNN/blob/main/CNNSpectroscopy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [110]:
import numpy as np
import random
from scipy.signal import savgol_filter
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
from math import perm
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import torch.optim as optim
from sklearn.model_selection import train_test_split

In [111]:
def add_gaussian_noise_df(df, noise_std=0.01):
    features = df.drop(columns=["label"])
    noisy_features = features + np.random.normal(0, noise_std, size=features.shape)
    result = noisy_features.copy()
    result["label"] = 1
    return result

def add_offset_df(df):
    features = df.drop(columns=["label"])
    training_std = features.values.std()
    offset = np.random.uniform(-0.1, 0.1) * training_std
    offset_features = features + offset
    result = offset_features.copy()
    result["label"] = 2
    return result

def apply_multiplication_df(df):
    features = df.drop(columns=["label"])
    training_std = features.values.std()
    factor = 1 + np.random.uniform(-0.1, 0.1) * training_std
    scaled_features = features * factor
    result = scaled_features.copy()
    result["label"] = 3
    return result

def flip_horizontal_df(df):
    features = df.drop(columns=["label"])
    flipped_features = features[features.columns[::-1]]
    result = flipped_features.copy()
    result["label"] = 4
    return result

def flip_vertical_df(df):
    features = df.drop(columns=["label"])
    mean_val = features.values.mean()
    flipped_features = 2 * mean_val - features
    result = flipped_features.copy()
    result["label"] = 5
    return result

def permute_segments_df(df, m=4):
    features = df.drop(columns=["label"])
    permuted_features = []
    for _, row in features.iterrows():
        segments = np.array_split(row.values, m)
        np.random.shuffle(segments)
        permuted_row = np.concatenate(segments)
        permuted_features.append(permuted_row)
    permuted_df = pd.DataFrame(permuted_features, columns=features.columns)
    permuted_df["label"] = 6
    return permuted_df

In [112]:
class SharedFeatureExtractor(nn.Module):
    def __init__(self, input_channels=1):
        super(SharedFeatureExtractor, self).__init__()
        self.block1 = nn.Sequential(
            nn.Conv1d(input_channels, 32, kernel_size=3, padding=1),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Conv1d(32, 32, kernel_size=3, padding=1),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2)
        )
        self.block2 = nn.Sequential(
            nn.Conv1d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Conv1d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2)
        )
        self.block3 = nn.Sequential(
            nn.Conv1d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Conv1d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2)
        )

    def forward(self, x):
        # x shape: (batch_size, 1, sequence_length)
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = torch.flatten(x, 1)  # flatten to (batch_size, features)
        return x

class TaskHead(nn.Module):
    def __init__(self, input_dim):
        super(TaskHead, self).__init__()
        self.classifier = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 1)  # Output is a single logit for binary classification
        )

    def forward(self, x):
        return self.classifier(x)

class MultiTaskTransformationRecognition(nn.Module):
    def __init__(self, sequence_length):
        super(MultiTaskTransformationRecognition, self).__init__()
        self.feature_extractor = SharedFeatureExtractor()

        # Determine the output dimension of the shared layers using a dummy input
        dummy_input = torch.randn(1, 1, sequence_length)
        with torch.no_grad():
            feat_dim = self.feature_extractor(dummy_input).shape[1]

        # Create 7 task-specific heads
        self.task_heads = nn.ModuleList([TaskHead(feat_dim) for _ in range(7)])

    def forward(self, x):
        shared_features = self.feature_extractor(x)  # shape: (batch_size, feature_dim)
        outputs = [head(shared_features) for head in self.task_heads]
        # Stack outputs into shape (batch_size, 7)
        return torch.cat(outputs, dim=1)

class ClassificationDataset(torch.utils.data.Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)  # <- must be Long for CrossEntropyLoss

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

class SpectralDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)  # float for BCEWithLogitsLoss

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

class SpectralTransformationDataset(Dataset):
    def __init__(self, spectra, labels):
        self.spectra = torch.tensor(spectra, dtype=torch.float32)  # (N, 228)
        self.labels = torch.tensor(labels, dtype=torch.float32)    # (N, 7)

    def __len__(self):
        return len(self.spectra)

    def __getitem__(self, idx):
        x = self.spectra[idx].unsqueeze(0)
        y = self.labels[idx]
        return x, y

class TransformationRecognitionCNN(nn.Module):
    def __init__(self, input_length):
        super(TransformationRecognitionCNN, self).__init__()

        # Shared convolutional layers
        self.shared_conv = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=32, kernel_size=3, padding=1),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Conv1d(in_channels=32, out_channels=32, kernel_size=3, padding=1),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2),

            nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Conv1d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2),

            nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Conv1d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2)
        )

        # Calculate flattened feature size
        with torch.no_grad():
            dummy_input = torch.zeros(1, 1, input_length)
            flattened_size = self.shared_conv(dummy_input).view(1, -1).shape[1]

        # Seven task-specific binary classification heads
        self.heads = nn.ModuleList([
            nn.Sequential(
                nn.Linear(flattened_size, 128),
                nn.ReLU(),
                nn.Dropout(0.5),
                nn.Linear(128, 64),
                nn.ReLU(),
                nn.Dropout(0.5),
                nn.Linear(64, 1)  # Output a single logit per head
            )
            for _ in range(7)
        ])

    def forward(self, x):
        # Input shape: (batch_size, num_features)
        x = x.unsqueeze(1)  # Convert to (batch_size, 1, num_features)
        features = self.shared_conv(x)
        features = features.view(features.size(0), -1)  # Flatten

        outputs = [head(features) for head in self.heads]  # List of (batch_size, 1)
        outputs = torch.cat(outputs, dim=1)  # Shape: (batch_size, 7)

        return outputs

class SpectralClassifier(nn.Module):
    def __init__(self, pretrained_shared_layers, input_length, num_classes):
        super(SpectralClassifier, self).__init__()
        self.shared_conv = pretrained_shared_layers

        # Get flattened size by running dummy input through shared layers
        with torch.no_grad():
            dummy_input = torch.zeros(1, 1, input_length)
            flattened_size = self.shared_conv(dummy_input).view(1, -1).shape[1]

        # New classifier head
        self.classifier = nn.Sequential(
            nn.Linear(flattened_size, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, num_classes)  # No activation, raw logits
        )

    def forward(self, x):
        x = x.unsqueeze(1)  # Add channel dim: (batch_size, 1, input_length)
        features = self.shared_conv(x)
        features = features.view(features.size(0), -1)
        logits = self.classifier(features)
        return logits

In [113]:
def prepare_transformation_dataset(
    raw_data, noise_data, offset_data, mult_data,
    hflip_data, vflip_data, perm_data
):
    # Combine data into one array
    all_data = np.concatenate([
        raw_data,
        noise_data,
        offset_data,
        mult_data,
        hflip_data,
        vflip_data,
        perm_data,
    ], axis=0)

    num_samples = raw_data.shape[0]
    total_samples = all_data.shape[0]

    # Create one-hot labels: (total_samples, 7)
    labels = np.zeros((total_samples, 7), dtype=np.float32)
    for i in range(7):
        labels[i * num_samples:(i + 1) * num_samples, i] = 1

    labels = all_data.get('label')
    # Shuffle data and labels in unison
    indices = np.arange(total_samples)
    np.random.shuffle(indices)

    all_data = all_data[indices]
    labels = labels[indices]

    return all_data, labels

In [114]:
# Path to your CSV file
csv_path = '/content/Dataset_RawSpectrum_NIRS_for_Intact_Mangoes.csv'  # Change this to your actual file path

# Load the CSV
# Assumes:
# - Each row = 1 sample
# - Each column = reflectance at one wavelength (e.g.,
df = pd.read_csv(csv_path)

# Drop non-numeric columns if necessary
df = df.select_dtypes(include=[np.number])

# Convert to NumPy array of shape (num_samples
raw_data = df.copy()
raw_data['label'] = 0

In [115]:
noise1 = add_gaussian_noise_df(df, noise_std=0.01)
noise2 = add_offset_df(df)
noise3 = apply_multiplication_df(df)
noise4 = flip_horizontal_df(df)
noise5 = flip_vertical_df(df)
noise6 = permute_segments_df(df, m=4)

all_data = pd.concat([raw_data,noise1,noise2,noise3,noise4,noise5,noise6])#noise1.append(noise2).append(noise3).append(noise4).append(noise5).append(noise6)
all_data = all_data.sample(frac=1, random_state=42)

X = all_data.drop(columns=['label']).values  # shape: (num_samples, num_features)
y = pd.get_dummies(all_data['label']).values
X_classifier = df.drop(columns=['label']).values
y_classifier = df['label'].values

In [116]:
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
X_classifier_scaled = scaler.fit_transform(X_classifier)

dataset = SpectralDataset(X_scaled, y)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [117]:
def train_model_with_validation(model, train_loader, val_loader, num_epochs=20, lr=0.0001, device='cuda' if torch.cuda.is_available() else 'cpu'):
    model = model.to(device)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(num_epochs):
        # === Train ===
        model.train()
        train_loss = 0.0
        for batch_X, batch_y in train_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)

            optimizer.zero_grad()
            outputs = model(batch_X)  # shape: (batch_size, 7)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * batch_X.size(0)

        avg_train_loss = train_loss / len(train_loader.dataset)

        # === Validate ===
        model.eval()
        correct = torch.zeros(7, device=device)
        total = torch.zeros(7, device=device)

        with torch.no_grad():
            for val_X, val_y in val_loader:
                val_X, val_y = val_X.to(device), val_y.to(device)
                outputs = model(val_X)
                preds = torch.sigmoid(outputs) > 0.5  # binary predictions

                correct += (preds == val_y.bool()).sum(dim=0)
                total += val_y.size(0)

        per_task_acc = (correct / total).cpu().numpy()
        acc_str = " | ".join([f"Task {i}: {acc:.3f}" for i, acc in enumerate(per_task_acc)])

        print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {avg_train_loss:.4f}")
        print(f"Validation Accuracy per Task: {acc_str}\n")

    print("Training complete.")
    return model


In [118]:
# 1. Assuming you already have:
#    - `X_scaled`: numpy array of shape (num_samples, num_features)
#    - `y_onehot`: one-hot numpy array of shape (num_samples, 7)

# 2. Split into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42, shuffle=True)

# 4. Create datasets and dataloaders
train_dataset = SpectralDataset(X_train, y_train)
val_dataset = SpectralDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [ ]:
# Split your dataset into train/val before creating DataLoaders
model = TransformationRecognitionCNN(input_length=X.shape[1])
trained_model = train_model_with_validation(model, train_loader, val_loader, num_epochs=50)

In [124]:
num_classes = 4
input_length = X.shape[1]  # number of wavelengths

# Extract pretrained shared layers
pretrained_shared = trained_model.shared_conv

# Optionally freeze them to preserve learned features
for param in pretrained_shared.parameters():
    param.requires_grad = False  # or True if you want to fine-tune them

# Create classifier model
classifier_model = SpectralClassifier(pretrained_shared, input_length, num_classes)

In [125]:
import matplotlib.pyplot as plt

def train_classifier(model, train_loader, val_loader, num_epochs=20, lr=0.0001, device='cuda' if torch.cuda.is_available() else 'cpu'):
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)

    train_losses = []
    val_accuracies = []

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for X_batch, y_batch in train_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device).long()

            optimizer.zero_grad()
            logits = model(X_batch)
            loss = criterion(logits, y_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item() * X_batch.size(0)

        avg_loss = total_loss / len(train_loader.dataset)
        train_losses.append(avg_loss)

        # Validation
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for X_val, y_val in val_loader:
                X_val = X_val.to(device)
                y_val = y_val.to(device)
                preds = model(X_val).argmax(dim=1)
                correct += (preds == y_val).sum().item()
                total += y_val.size(0)
        acc = correct / total
        val_accuracies.append(acc)

        print(f"Epoch {epoch+1}/{num_epochs} - Loss: {avg_loss:.4f}, Val Acc: {acc:.4f}")

    return model, train_losses, val_accuracies


In [ ]:
# y_labels should be shape (num_samples,) with integers like 0, 1, 2
X_train, X_val, y_train, y_val = train_test_split(X_classifier_scaled, y_classifier, test_size=0.2, random_state=42)

train_dataset = ClassificationDataset(X_train, y_train)
val_dataset = ClassificationDataset(X_val, y_val)

train_classification_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_classification_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
print("Unique class labels:", np.unique(y_classifier))
trained_classifier, train_losses, val_accuracies = train_classifier(classifier_model, train_classification_loader, val_classification_loader, num_epochs=300)


In [ ]:


# Plotting
plt.figure(figsize=(12, 5))

# Loss
plt.subplot(1, 2, 1)
plt.plot(train_losses, marker='o')
plt.title("Training Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")

# Accuracy
plt.subplot(1, 2, 2)
plt.plot(val_accuracies, marker='o', color='green')
plt.title("Validation Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")

plt.tight_layout()
plt.show()
